In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q Flask ibm-watson ibm-db

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 19.4 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/templates

/content/drive/MyDrive/templates


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://uy0g1rpjixp-496ff2e9c6d22116-8000-colab.googleusercontent.com/
https://uy0g1rpjixp-496ff2e9c6d22116-8000-colab.googleusercontent.com/


# Database

In [ ]:
if __name__ == '__main__':
    app.run(port = 8000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:15] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:16] "GET /image.png?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:16] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:17] "GET /update?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:29] "POST /submit?authuser=0 HTTP/1.1" 200 -


2004-11-11


INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:31] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:32] "GET /image.png?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:32] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:55:33] "GET /update?authuser=0 HTTP/1.1" 200 -


In [ ]:
from flask import Flask, render_template, request, jsonify, Response
import ibm_db
import pandas as pd
from datetime import date
import os

# Set your IBM Db2 credentials in the environment variables or manually assign them in the code below.
dsn_hostname = os.getenv('DB_HOST', r'815fa4db-dc03-4c70-869a-a9cc13f33084.bs2io90l08kqb1od8lcg.databases.appdomain.cloud') # change this
dsn_uid = os.getenv('DB_UID', 'bxw40273')  # change this
dsn_pwd = os.getenv('DB_PWD', 'OIHuQ19w0Q7j2iBI')  # change this
dsn_port = os.getenv('DB_PORT', '30367')  # change this
dsn_database = "bludb"  # change if necessary
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

# Create the dsn connection string
conn_str = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)


In [ ]:
app = Flask(__name__, template_folder = "/content/drive/MyDrive/templates")

@app.route('/')
def index():
    return render_template("index.html")


In [ ]:
@app.route('/submit', methods=['POST'])
def submit():
    try:
      name = request.form['name']
      birthday = request.form['birthday']
      user_id = request.form['id']
      major = request.form["major"]

      # Connect to the database
      conn = ibm_db.connect(conn_str, '', '')

      # Insert data into the database
      sql = "INSERT INTO STUDENTS VALUES (?, ?, ?, ?)"
      stmt = ibm_db.prepare(conn, sql)
      ibm_db.bind_param(stmt, 1, name)
      ibm_db.bind_param(stmt, 2, birthday)
      ibm_db.bind_param(stmt, 3, user_id)
      ibm_db.bind_param(stmt, 4, major)
      ibm_db.execute(stmt)

      print(birthday)
      # Close the connection
      ibm_db.close(conn)

    except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")

    return index()



In [ ]:
@app.route('/delete', methods=['POST'])
def delete():
  try:
      user_id = request.form['id']

      # Connect to the database
      conn = ibm_db.connect(conn_str, '', '')

      # Delete data from the database
      sql = "DELETE FROM STUDENTS WHERE \"MA_SV\" = ?"
      stmt = ibm_db.prepare(conn, sql)
      ibm_db.bind_param(stmt, 1, user_id)
      ibm_db.execute(stmt)

      # Close the connection
      ibm_db.close(conn)

  except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")
  return index()



In [ ]:
@app.route('/update')
def update():
    conn = ibm_db.connect(conn_str, "", "")

    # Query to get the first 5 rows from the table
    command = "SELECT * FROM STUDENTS ORDER BY \"MA_SV\" DESC"
    stmt = ibm_db.exec_immediate(conn, command)

    # Fetch the result set
    result = ibm_db.fetch_both(stmt)
    rows = []

    # Loop through the result and fetch the rows
    while result:
        rows.append({k:v for (k,v) in result.items() if not isinstance(k, int)})
        result = ibm_db.fetch_both(stmt)

    # Convert result to a pandas DataFrame for better readability
    return pd.DataFrame(rows).to_html()